In [1]:
pip install cvzone


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Bibliotecas a serem utilizada no projeto
import pandas as pd
import getpass
import random
import cv2
import numpy as np
import cvzone

In [3]:
#Conversão e leitura de um arquivo .csv para dataframe utilizando o biblioteca pandas
pilotos = pd.read_csv("/Users/diegoc85/Challenge_python/formula_e_pilotos_2023_2024.csv", header = None)
pilotos

,0,1
0,Equipe,Piloto
1,Avalanche Andretti Formula E Team,Jake Dennis
2,Avalanche Andretti Formula E Team,André Lotterer
3,Envision Racing,Sébastien Buemi
4,Envision Racing,Nick Cassidy
5,Jaguar TCS Racing,Mitch Evans
6,Jaguar TCS Racing,Sam Bird
7,DS Penske,Stoffel Vandoorne
8,DS Penske,Jean-Éric Vergne
9,Nissan Formula E Team,Norman Nato


In [5]:
usuarios = []

#Cadastrar um novo usuario no app para acesso do visão em tempo real
def cadastrar_usuarios(usuarios):
    print("---- CADASTRO ----")
    nome = input("Digite seu nome:")
    email = input("Digite seu e-mail")
    senha = getpass.getpass("Digite sua senha:")
    usuarios.append({'Nome':nome, 'E-mail':email, 'Senha':senha})
    print('Cadastro realizado com sucesso!\n')
    print("-----------------------------------------")

    for indice, dicionario in enumerate(usuarios):
        print(f"\nCadastro {indice + 1}:")
        for chave, valor in dicionario.items():
            print(f"    {chave} : {valor}")
    print("\n")

      
# Logar na área exclusiva do Vision FE
def login(usuarios):
    tentativas = 3

    while tentativas > 0:
        print("----- LOGIN ----")
        email = input("E-mail:")
        senha = getpass.getpass('Senha:')
        for usuario in usuarios:
            if usuario['E-mail'] == email and usuario['Senha'] == senha:
                print(f"Login efetuado com sucesso, {usuario["Nome"]}! ")
                return True
            
            else:
                tentativas -= 1
                print(f"E-mail ou Senha incorretos. Tente novamente! Você tem mais {tentativas}!")
    print("Tentativas excedidas!Fim do programa.")
    return False

def menu_exclusivo(pilotos):
    print("\n---- Bem-vindo à Área Exclusiva do Vision FE! ----")

    while True:
        print("\n1) Acesse o Vision FE")
        print("2) Mostrar velocidade atual do carro rastreado")
        print("3) Posição atual do carro rastreado")
        print("4) Nome do piloto e da equipe do carro rastreado")
        print("5) Sair")

        opcao = input("\nEscolha uma das opções do Menu.\n")

        match opcao:
            case "1":
                print("\n\nAcessando o Vision FE\n\n")

                videoPath = 'example2.mp4' #video em que o objeto a ser rastreado aparece
                configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt' #config do modelo
                modelPath = 'frozen_inference_graph.pb'
                classesPath = 'coco.names'

                net = cv2.dnn_DetectionModel(modelPath,configPath)

                net.setInputSize(320,320)
                net.setInputScale(1.0/127.5)
                net.setInputMean((127.5,127.5,127.5))
                net.setInputSwapRB(True)


                with open(classesPath,'r') as f:
                    classesList = f.read().splitlines()

                video = cv2.VideoCapture(videoPath)

                while True:
                    check,img = video.read()
                    img = cv2.resize(img,(1270,720))

                    labels, confs, bboxs = net.detect(img,confThreshold=0.5)

                    bboxs = list(bboxs)
                    confs = list(np.array(confs).reshape(1,-1)[0])
                    confs = list(map(float,confs))

                    bboxsIdx = cv2.dnn.NMSBoxes(bboxs,confs,score_threshold=0.5, nms_threshold=0.3)

                    if len(bboxsIdx) !=0:
                        for x in range(0,len(bboxsIdx)):
                            bbox = bboxs[np.squeeze(bboxsIdx[x])]
                            conf = confs[np.squeeze(bboxsIdx[x])]
                            labelsID = np.squeeze(labels[np.squeeze(bboxsIdx[x])])-1
                            label = classesList[labelsID]

                            print(bbox,labelsID,conf)
                            x,y,w,h = bbox

                            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
                            cvzone.putTextRect(img,f'{label} {round(conf,2)}',(x,y-10),colorR=(255,0,0),scale=1,thickness=2)

                    cv2.imshow('Imagem',img)

                    #if cv2.waitKey(1)==27:
                    if cv2.waitKey(1) & 0xFF == ord('q'): #finalizar o programa pressionando a tecla q
                        break

            case "2":
                velocidade = random.randint(0,320)
                print(f"\n\nA velocidade atual do carro rastreado é de {velocidade} km/h.\n\n")
            case "3":
                posicao = random.randint(1,22)
                print(f"\n\nO carro rastreado está na posição {posicao} na corrida.\n\n")
            case "4":
                piloto_equipe = pilotos.sample()
                piloto = piloto_equipe.iloc[0,1]
                equipe = piloto_equipe.iloc[0,0]
                print(f"\n\nO nome da equipe é {equipe} e o piloto é o {piloto}.\n\n")
            case "5":
                print("\nSair do programa. Até a próxima!\n")
                break
            case _:
                print("\nOpção Inválida. Tente novamente!\n")

# Tela de BOAS-VINDAS
def boas_vindas():
    print("\n ---- BEM-VINDO AO VISION FE ----")
    print("\n ---- Faça seu cadastro ----\n")

    acao = input("\n Digite 'c' para fazer seu cadastro.")

    if acao == 'c' or acao == 'C':
        cadastrar_usuarios(usuarios)
        if login(usuarios):
            menu_exclusivo(pilotos)
    else:
        print("Opção inválida. Fim do programa!")


boas_vindas()



    




 ---- BEM-VINDO AO VISION FE ----

 ---- Faça seu cadastro ----

---- CADASTRO ----
Cadastro realizado com sucesso!

-----------------------------------------

Cadastro 1:
    Nome : Diego
    E-mail : diego
    Senha : diego


----- LOGIN ----
Login efetuado com sucesso, Diego! 

---- Bem-vindo à Área Exclusiva do Vision FE! ----

1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


A velocidade atual do carro rastreado é de 301 km/h.



1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


O carro rastreado está na posição 21 na corrida.



1) Acesse o Vision FE
2) Mostrar velocidade atual do carro rastreado
3) Posição atual do carro rastreado
4) Nome do piloto e da equipe do carro rastreado
5) Sair


O nome da equipe é Envision Racing e o piloto é o Nick Cassidy.



: 